In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import zipfile
from tqdm import tqdm

In [2]:
import logging
logging.getLogger('tensorflow').disabled = True

In [3]:
with zipfile.ZipFile('archive.zip', mode='r') as zip_file:
    for zf in tqdm(zip_file.infolist(), desc="Extracting"):
        try:
            zip_file.extract(zf)                
        except Exception as e:
            print(e)
zip_file.close()

Extracting: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 121.22it/s]


### Trying to read sql tabel to pandas dataframe

In [4]:
import sqlite3

In [5]:
con = sqlite3.connect('database.sqlite')

In [6]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [7]:
tables

[('Iris',)]

In [8]:
df = pd.read_sql_query("SELECT * FROM Iris" , con, index_col="Id")
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


In [10]:
df.Species = df.Species.astype("category")

In [11]:
category = pd.get_dummies(df.Species)
category

Species,Iris-setosa,Iris-versicolor,Iris-virginica
Id,,,
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
...,...,...,...
146,0,0,1
147,0,0,1
148,0,0,1


In [12]:
new_df = pd.concat([df, category], axis=1)
new_df = new_df.drop(columns='Species')
new_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
Id,,,,,,,
1,5.1,3.5,1.4,0.2,1,0,0
2,4.9,3.0,1.4,0.2,1,0,0
3,4.7,3.2,1.3,0.2,1,0,0
4,4.6,3.1,1.5,0.2,1,0,0
5,5.0,3.6,1.4,0.2,1,0,0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [14]:
target = ['Iris-setosa','Iris-versicolor','Iris-virginica']
feature = [i for i in new_df.columns if i not in target]

In [15]:
X = new_df[feature]
y = new_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120, 3), (30, 3))

In [16]:
minmax = MinMaxScaler()
minmax.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [17]:
X_train = minmax.transform(X_train)
X_test = minmax.transform(X_test)

In [18]:
import numpy as np
  
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [19]:
model = keras.Sequential([
        keras.layers.Dense(units=64, activation="relu", input_shape=(4,)),
        keras.layers.Dense(units=64, activation="relu"),
        keras.layers.Dense(units=3, activation="softmax")
])

In [20]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [21]:
model.fit(X_train, y_train, epochs=120)

Train on 120 samples
Epoch 1/120
120/120 [==============================] - 2s 15ms/sample - loss: 1.0915 - accuracy: 0.6083
Epoch 2/120
120/120 [==============================] - 0s 241us/sample - loss: 1.0403 - accuracy: 0.6750
Epoch 3/120
120/120 [==============================] - 0s 216us/sample - loss: 0.9906 - accuracy: 0.6750
Epoch 4/120
120/120 [==============================] - 0s 270us/sample - loss: 0.9497 - accuracy: 0.6583
Epoch 5/120
120/120 [==============================] - 0s 266us/sample - loss: 0.9143 - accuracy: 0.6417
Epoch 6/120
120/120 [==============================] - 0s 258us/sample - loss: 0.8790 - accuracy: 0.6667
Epoch 7/120
120/120 [==============================] - 0s 216us/sample - loss: 0.8430 - accuracy: 0.6583
Epoch 8/120
120/120 [==============================] - 0s 133us/sample - loss: 0.8060 - accuracy: 0.6583
Epoch 9/120
120/120 [==============================] - 0s 208us/sample - loss: 0.7703 - accuracy: 0.6583
Epoch 10/120
120/120 [=============

120/120 [==============================] - 0s 141us/sample - loss: 0.0960 - accuracy: 0.9750
Epoch 79/120
120/120 [==============================] - 0s 116us/sample - loss: 0.0941 - accuracy: 0.9750
Epoch 80/120
120/120 [==============================] - 0s 116us/sample - loss: 0.0919 - accuracy: 0.9750
Epoch 81/120
120/120 [==============================] - 0s 125us/sample - loss: 0.0903 - accuracy: 0.9667
Epoch 82/120
120/120 [==============================] - 0s 133us/sample - loss: 0.0902 - accuracy: 0.9750
Epoch 83/120
120/120 [==============================] - 0s 125us/sample - loss: 0.0907 - accuracy: 0.9667
Epoch 84/120
120/120 [==============================] - 0s 116us/sample - loss: 0.0868 - accuracy: 0.9750
Epoch 85/120
120/120 [==============================] - 0s 175us/sample - loss: 0.0860 - accuracy: 0.9667
Epoch 86/120
120/120 [==============================] - 0s 158us/sample - loss: 0.0853 - accuracy: 0.9667
Epoch 87/120
120/120 [==============================] - 0s 

In [22]:
model.evaluate(X_test, y_test)

30/30 [==============================] - 0s 5ms/sample - loss: 0.0735 - accuracy: 0.9667


[0.07347183674573898, 0.96666664]

In [23]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [24]:
df.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa


In [25]:
x_val = np.array([[4, 2, 4, 1]])

In [26]:
x_val.shape

(1, 4)

In [30]:
x_val = minmax.transform(x_val)
x_val

array([[-1.29065744, -0.83333333, -0.08310249,  0.11458333]])

In [31]:
y_val = model.predict(x_val)
y_val

array([[0.6727908 , 0.3196668 , 0.00754238]], dtype=float32)

In [32]:
pred = np.argmax(y_val)
pred

0

`Artinya termasuk kelas iris-sentosa`